In [2]:
import pandas as pd
test = pd.read_csv("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/processed/test.csv")
train = pd.read_csv("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/processed/training.csv")
frames=[test,train]
data = pd.concat(frames)

In [3]:
f = open("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/vocabbuilder.txt","w",encoding='utf-8')
for index,row in data.iterrows():
    #print(row["text"])
    if row["text"] == "":
        continue
    #text = " ".join(str(row["text"]))
    f.write(str(row["text"])+"\n")
f.close()

### build vocab file and calculate appearence-fraction of words

In [4]:
import nltk

def get_frac_dist(token_list):
    '''
    Computes frequency count and fraction of individual words in a list.

    Parameters
    ----------
    token_list : list
        List of all (non-unique) tokens from some corpus

    Returns
    -------
    dict
        Dictionary of { token, (count, fraction) } pairs.
    '''

    token_list = nltk.word_tokenize(content)
    total_token_count = len(token_list)

    freq_dict = nltk.FreqDist(token_list)

    frac_dict = {}
    for token, count in freq_dict.items():
        frac_dict[token] = (count, count / total_token_count)

    return frac_dict

In [5]:
with open("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/vocabbuilder.txt") as f:
    content = f.read().lower()

token_list = nltk.word_tokenize(content.lower())
frac_dist = get_frac_dist(token_list)

In [6]:
i = 0
for k, v in frac_dist.items():

    i += 1
    if i > 10:
        break

    token = k.lower()
    count = v[0]
    frac = v[1]
    print('%-20s: %5d, %0.5f' % (token, count, frac))

need                :  5500, 0.00217
info                :  1410, 0.00056
bonneville          :    10, 0.00000
organization        : 18860, 0.00745
university          :  9410, 0.00372
buffalo             :   262, 0.00010
lines               : 15278, 0.00604
newssoftware        :   308, 0.00012
vaxvms              :   310, 0.00012
vnews               :   275, 0.00011


In [7]:
l = [ (k, v[0], v[1]) for k, v in frac_dist.items() if v[0] > 300]
print(len(l))

1469


In [8]:
len(frac_dist)

144591

In [9]:
print(l[0][0])

need


In [10]:
vocab = set()
i = 0
for i in range(len(l)):
    #print(word)
    vocab.add(l[i][0])
vocab = sorted(vocab)
print(len(vocab))

1469


In [11]:
vocab[1]

'able'

In [12]:
with open("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/vocab_docnade.vocab", "w") as f:
    for item in vocab:
        f.write(item + "\n")
f.close()  

In [16]:
import pandas as pd

datatrain = pd.read_csv('/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/processed/training.csv') 
datatest = pd.read_csv('/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/processed/test.csv') 

In [17]:
import spacy
import nltk
from nltk.tokenize import word_tokenize

In [18]:
# remove stopwords and only keep words that are in vocab

def remove_stopwords(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw= [word for word in text_tokens if not word in all_stopwords and word in vocab]
    return tokens_without_sw

sp = spacy.load('en_core_web_lg')
all_stopwords = sp.Defaults.stop_words


datatrain["text"] = datatrain['text'].apply(remove_stopwords)
datatest["text"] = datatest['text'].apply(remove_stopwords)
datatrain['length']=datatrain['text'].apply(len)
datatest['length']=datatest['text'].apply(len)


In [19]:
label_dict = {
    "0": 'alt.atheism',
     "1":'comp.graphics',
     "2":'comp.os.ms-windows.misc',
     "3":'comp.sys.ibm.pc.hardware',
     "4":'comp.sys.mac.hardware',
     "5":'comp.windows.x',
     "6":'misc.forsale',
     "7":'rec.autos',
     "8":'rec.motorcycles',
     "9":'rec.sport.baseball',
     "10":'rec.sport.hockey',
     "11":'sci.crypt',
     "12":'sci.electronics',
     "13":'sci.med',
     "14":'sci.space',
     "15":'soc.religion.christian',
     "16":'talk.politics.guns',
     "17":'talk.politics.mideast',
     "18":'talk.politics.misc',
     "19":'talk.religion.misc'
}

In [20]:
datatrain["label"]=""
for index, row in datatrain.iterrows():
    datatrain["label"][index] = label_dict[str(row["target"])]
datatest["label"]=""
for index, row in datatest.iterrows():
    datatest["label"][index] = label_dict[str(row["target"])]    
    

datatrain.head()

<ipython-input-20-3591ac13fd2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatrain["label"][index] = label_dict[str(row["target"])]
<ipython-input-20-3591ac13fd2c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatest["label"][index] = label_dict[str(row["target"])]


,text,target,length,label
0,"[car, wonder, car, day, door, sport, car, look...",7,30,rec.autos
1,"[final, clock, report, fair, number, soul, upg...",4,37,comp.sys.mac.hardware
2,"[question, usa, folk, mac, plus, finally, star...",4,92,comp.sys.mac.hardware
3,"[world, version, write, write, article, know, ...",1,25,comp.graphics
4,"[article, article, write, clear, memory, error...",14,44,sci.space


In [21]:
frames = [datatrain,datatest]
data = pd.concat(frames)
data.head()

,text,target,length,label
0,"[car, wonder, car, day, door, sport, car, look...",7,30,rec.autos
1,"[final, clock, report, fair, number, soul, upg...",4,37,comp.sys.mac.hardware
2,"[question, usa, folk, mac, plus, finally, star...",4,92,comp.sys.mac.hardware
3,"[world, version, write, write, article, know, ...",1,25,comp.graphics
4,"[article, article, write, clear, memory, error...",14,44,sci.space


In [24]:
# split for evaluation set
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(datatrain, test_size=0.2, random_state=42)

In [25]:
print(len(datatest),len(data),len(train_data),len(val_data))

7532 18846 9051 2263


In [26]:
data.mean(axis = 0, skipna = True) 

target     9.293166
length    61.153720
dtype: float64

In [27]:
data.std(axis = 0, skipna = True) 

target      5.562798
length    131.508880
dtype: float64

In [28]:
datatest.mean(axis = 0, skipna = True) 

target     9.293415
length    58.784918
dtype: float64

In [29]:
datatest.std(axis = 0, skipna = True) 

target      5.563286
length    118.100050
dtype: float64

In [35]:
train_data.mean(axis = 0, skipna = True) 

target     9.267926
length    63.100210
dtype: float64

In [36]:
train_data.std(axis = 0, skipna = True) 

target      5.558124
length    142.415676
dtype: float64

In [37]:
val_data.mean(axis = 0, skipna = True) 

target     9.393283
length    61.252762
dtype: float64

In [38]:
val_data.std(axis = 0, skipna = True) 

target      5.581166
length    128.320115
dtype: float64

In [39]:
## 20NGfile
f = open("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/20NG.txt","w",encoding='utf-8')
for index,row in data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()

In [40]:
## 20NG train txt
f = open("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/20NG_train.txt","w",encoding='utf-8')
for index,row in train_data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()



In [41]:
## 20NG val txt
f = open("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/20NG_val.txt","w",encoding='utf-8')
for index,row in val_data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()


In [42]:
## 20NG test txt
f = open("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/20NG_test.txt","w",encoding='utf-8')
for index,row in datatest.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(row["label"]+"\t"+text+"\n")
f.close()


In [43]:
## 20NG text only  txt
f = open("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/20NGTextonly.txt","w",encoding='utf-8')
for index,row in data.iterrows():
    if row["text"] == []:
        continue
    text = " ".join(row["text"])
    f.write(text+"\n")
f.close()

In [44]:
# test.csv
datatestcsv = datatest.copy()
for index,row in datatestcsv.iterrows():
    if row["text"] == []:
        continue
    datatestcsv["text"][index] = " ".join(row["text"])
datatestcsv.to_csv(path_or_buf="/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/test.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")


<ipython-input-44-251e3e68d315>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatestcsv["text"][index] = " ".join(row["text"])


In [45]:
# training.csv
datatraincsv = train_data.copy()
for index,row in datatraincsv.iterrows():
    if row["text"] == []:
        continue
    datatraincsv["text"][index] = " ".join(row["text"])
datatraincsv.to_csv(path_or_buf="/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/training.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")


<ipython-input-45-858a248a1b6a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatraincsv["text"][index] = " ".join(row["text"])


In [46]:
# validation.csv
datavalcsv = val_data.copy()
for index,row in datavalcsv.iterrows():
    if row["text"] == []:
        continue
    datavalcsv["text"][index] = " ".join(row["text"])
datavalcsv.to_csv(path_or_buf="/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/validation.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")


<ipython-input-46-3a6ba573720e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datavalcsv["text"][index] = " ".join(row["text"])


In [47]:
print(len(datatrain),len(datatest),len(data))

11314 7532 18846


In [48]:
word2id = dict()
i = 0
for word in vocab:
    word2id[word] = i
    i+=1

In [49]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
label_mapping = dict()
i = 0
for word in newsgroups_train.target_names:
    label_mapping[word]=i
    i+=1

In [50]:
#test_docnade.xlsx
with open("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/20NG_test.txt", "r") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
content= [x.split("\t") for x in content]
csvfile = pd.DataFrame(content,columns=["label","text"])
for index,row in csvfile.iterrows():
    id_string = str()
    for word in row["text"].split():
        id_string += " "+str(word2id[word])
    label_id = label_mapping[row["label"]]    
    csvfile["text"][index] = id_string
    csvfile["label"][index] = label_id
csvfile.to_csv(path_or_buf="/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/test_docnade.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")    

In [51]:
#training_docnade.xlsx
with open("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/20NG_train.txt", "r") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
content= [x.split("\t") for x in content]
csvfile = pd.DataFrame(content,columns=["label","text"])

for index,row in csvfile.iterrows():
    id_string = str()
    for word in row["text"].split():
        id_string += " "+str(word2id[word])
    label_id = label_mapping[row["label"]]    
    csvfile["text"][index] = id_string
    csvfile["label"][index] = label_id
csvfile.to_csv(path_or_buf="/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/training_docnade.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")

In [53]:
#validation_docnade.xlsx
with open("/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/20NG_val.txt", "r") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
content= [x.split("\t") for x in content]
csvfile = pd.DataFrame(content,columns=["label","text"])
csvfile.head()
for index,row in csvfile.iterrows():
    id_string = str()
    for word in row["text"].split():
        id_string += " "+str(word2id[word])
    label_id = label_mapping[row["label"]]    
    csvfile["text"][index] = id_string
    csvfile["label"][index] = label_id
csvfile.to_csv(path_or_buf="/home/flokla/Dropbox/Desktop/bachelorarbeit/realshit/Datasets/20NG_short/idocnade/validation_docnade.csv",columns=["label","text"],header=False,index=False,encoding="utf-8",line_terminator="\n")